# Spreadsheet

    Make a spreadsheet using pinkfish.  This is useful for developing trading strategies.
    It can also be used as a tool for buy and sell signals that you then manually execute.

In [1]:
import datetime

import matplotlib.pyplot as plt
import pandas as pd

from talib.abstract import *

import pinkfish as pf
import pinkfish.itable as itable

# Format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# Set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)
ts.tail()

,open,high,low,close,adj_close,volume
date,,,,,,
2023-12-21,471.33,472.98,468.84,472.70,472.70,86667500
2023-12-22,473.86,475.38,471.70,473.65,473.65,67126600
2023-12-26,474.07,476.58,473.99,475.65,475.65,55387000
2023-12-27,475.44,476.66,474.89,476.51,476.51,68000300
2023-12-28,476.88,477.55,476.26,476.69,476.69,77106300


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [5]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)
ts.head()

,open,high,low,close,adj_close,volume
date,,,,,,
1993-01-29,43.97,43.97,43.75,43.94,24.84,1003200
1993-02-01,43.97,44.25,43.97,44.25,25.02,480500
1993-02-02,44.22,44.38,44.12,44.34,25.07,201300
1993-02-03,44.41,44.84,44.38,44.81,25.34,529400
1993-02-04,44.97,45.09,44.47,45.00,25.44,531500


Add technical indicators

In [6]:
# Add 200 day MA.
ts['sma200'] = pf.SMA(ts, timeperiod=200)

# Add ATR.
ts['atr'] = ATR(ts, timeperiod=14)

# Add 5 day high, and 5 day low
ts['high5'] = pd.Series(ts.high).rolling(window=5).max()
ts['low5'] = pd.Series(ts.low).rolling(window=5).min()

# Add RSI, and 2-period cumulative RSI
ts['rsi2'] = RSI(ts, timeperiod=2)
ts['c2rsi2'] = pd.Series(ts.rsi2).rolling(window=2).sum()

# Add midpoint
ts['mp'] = (ts.high + ts.low) / 2

# Add 10 day SMA of midpoint
ts['sma10'] = pd.Series(ts.mp).rolling(window=10).mean()

# Add temporary rolling 10 day Standard Deviation of midpoint
ts['__sd__'] = pd.Series(ts.mp).rolling(window=10).std()

# Add standard deviation envelope or channel around midpoint
ts['upper'] = ts.sma10 + ts['__sd__']*2
ts['lower'] = ts.sma10 - ts['__sd__']*2

# Drop temporary columns.
ts.drop(columns=['__sd__'], inplace=True)

Finalize timeseries

In [7]:
ts, start = pf.finalize_timeseries(ts, start, dropna=True)
ts.tail()

,open,high,low,close,adj_close,volume,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,
2023-12-21,471.33,472.98,468.84,472.70,472.70,86667500,432.19,4.33,475.90,467.43,62.09,88.26,470.91,467.81,478.24,457.37
2023-12-22,473.86,475.38,471.70,473.65,473.65,67126600,432.60,4.28,475.90,467.82,68.63,130.72,473.54,469.26,478.18,460.35
2023-12-26,474.07,476.58,473.99,475.65,475.65,55387000,433.05,4.19,476.58,467.82,81.82,150.45,475.28,470.71,478.10,463.33
2023-12-27,475.44,476.66,474.89,476.51,476.51,68000300,433.50,4.01,476.66,467.82,86.65,168.47,475.78,472.05,477.27,466.82
2023-12-28,476.88,477.55,476.26,476.69,476.69,77106300,433.93,3.82,477.55,468.84,87.98,174.63,476.90,473.00,477.93,468.06


Select a smaller time from for use with itable

In [8]:
df = ts['2023-01-01':]
df.head()

,open,high,low,close,adj_close,volume,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,
2023-01-03,384.37,386.43,377.83,380.82,375.12,74850700,399.62,6.96,386.43,376.42,38.39,96.67,382.13,381.28,384.29,378.27
2023-01-04,383.18,385.88,380.00,383.76,378.01,85934100,399.31,6.88,386.43,376.42,72.58,110.96,382.94,381.47,384.65,378.29
2023-01-05,381.72,381.84,378.76,379.38,373.70,76970500,398.99,6.75,386.43,377.83,27.35,99.93,380.30,381.50,384.63,378.37
2023-01-06,382.61,389.25,379.41,388.08,382.27,104189600,398.68,6.97,389.25,377.83,79.10,106.45,384.33,381.43,384.21,378.65
2023-01-09,390.37,393.70,387.67,387.86,382.05,73978100,398.40,6.90,393.70,377.83,76.35,155.44,390.69,382.45,388.83,376.06


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [11]:
pt = itable.PrettyTable(
    df, tstyle=itable.TableStyle(theme='theme1'), header_row=True, rpt_header=20)

# pt = itable.PrettyTable(
#      df, tstyle=itable.TableStyle(theme='theme1'), header_row=True, rpt_header=20)

pt.update_col_header_style(
    format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(
    format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'].iloc[row] == pt.df['high'].iloc[row]) and \
       (pt.df['high5'].iloc[row] > pt.df['high'].iloc[row-1]):
        col = df.columns.get_loc('high5')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'].iloc[row] == pt.df['low'].iloc[row]) and \
       (pt.df['low5'].iloc[row] < pt.df['low'].iloc[row-1]):
        col = df.columns.get_loc('low5')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [12]:
pt

,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10,UPPER,LOWER
2023/01/03,384.37,386.43,377.83,380.82,375.12,74850700,399.62,6.96,386.43,376.42,38.39,96.67,382.13,381.28,384.29,378.27
2023/01/04,383.18,385.88,380.00,383.76,378.01,85934100,399.31,6.88,386.43,376.42,72.58,110.96,382.94,381.47,384.65,378.29
2023/01/05,381.72,381.84,378.76,379.38,373.70,76970500,398.99,6.75,386.43,377.83,27.35,99.93,380.30,381.50,384.63,378.37
2023/01/06,382.61,389.25,379.41,388.08,382.27,104189600,398.68,6.97,389.25,377.83,79.10,106.45,384.33,381.43,384.21,378.65
2023/01/09,390.37,393.70,387.67,387.86,382.05,73978100,398.40,6.90,393.70,377.83,76.35,155.44,390.69,382.45,388.83,376.06
2023/01/10,387.25,390.65,386.27,390.58,384.73,65358100,398.10,6.72,393.70,378.76,87.28,163.63,388.46,383.24,390.48,375.99
2023/01/11,392.23,395.60,391.38,395.52,389.60,68881100,397.82,6.60,395.60,378.76,95.25,182.53,393.49,384.45,394.00,374.89
2023/01/12,396.67,398.49,392.42,396.96,391.02,90157700,397.52,6.56,398.49,379.41,96.52,191.78,395.46,386.00,397.19,374.81
2023/01/13,393.62,399.10,393.34,398.50,392.53,63903900,397.21,6.51,399.10,386.27,97.79,194.31,396.22,387.45,399.86,375.04
2023/01/17,398.48,400.23,397.06,397.77,391.81,62677300,396.90,6.27,400.23,386.27,72.70,170.49,398.65,389.27,402.45,376.09
